In [1]:
# # Mounting the Google Drive
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# import zipfile
# with zipfile.ZipFile("EOphtha_10k_Dataset.zip","r") as zip_ref:
#     zip_ref.extractall("DataSet")

In [3]:
# !pip install patchify

In [4]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
# from patchify import patchify
from natsort import natsorted
# from patchify import unpatchify
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout, Lambda, Add, Multiply

In [6]:
def simAM(inp1, inp2):

    lamda = 1e-4
    d = K.square(inp1 - K.mean(inp1, axis=[1,2], keepdims=True))
    n = (inp1.shape[1] * inp1.shape[2]) - 1
    v = K.sum(d, axis=[1,2], keepdims=True) / n
    td_weights = (d / (4*(v + lamda))) + 0.5
    td_weights = Activation('sigmoid')(td_weights)
    out = Multiply()([inp2, td_weights])

    return out, td_weights

In [7]:
# A Convolutional Block is composed of 2 consecutive (conv + BN + relu) operations, with a BN in between.

def conv_block(inp, num_filters):

    x = Conv2D(num_filters, (3,3), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [8]:
# A Encoder Block is combination of conv_block & max pooling operation.

def encoder_block(inp, num_filters):

    x = conv_block(inp, num_filters)
    p = MaxPool2D((2,2))(x)

    return x,p

In [9]:
# Defining Decoder Block
def decoder_block(inp1, inp2, num_filters):

    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inp1)
    skip_features, d3_weights = simAM(x, inp2)

    x = Concatenate()([x, skip_features])
    x = Dropout(0.2)(x)

    x = Conv2D(num_filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x, d3_weights

In [10]:
def build_UNet(inp1_shape,inp2_shape):

    inps1 = Input(inp1_shape)
    inps2 = Input(inp2_shape)

    s = Lambda(lambda k:k/255)(inps1)
    c1, p1 = encoder_block(s, 32)
    p1 = Dropout(0.2)(p1)
    c2, p2 = encoder_block(p1, 64)
    p2 = Dropout(0.2)(p2)
    c3, p3 = encoder_block(p2, 128)
    p3 = Dropout(0.2)(p3)
    c4, p4 = encoder_block(p3, 256)
    p4 = Dropout(0.2)(p4)

    c5 = conv_block(p4, 512)

    u6, d34_weights = decoder_block(c5, c4, 256)
    u7, d33_weights = decoder_block(u6, c3, 128)
    u8, d32_weights = decoder_block(u7, c2, 64)
    u9, d31_weights = decoder_block(u8, c1, 32)

    output1 = Conv2D(1, (1,1), padding="same", activation="sigmoid")(u9)
    output2 = K.min((K.sum((inps2 * K.square(d31_weights)), axis=[1,2]) / (K.sum(inps2, axis=[1,2]) + K.epsilon())),axis=[1])

    model = Model(inputs=[inps1,inps2], outputs=[output1,output2], name="UNet")

    return model

In [11]:
# Here, Images Details are to be Included
Img_Width = 48
Img_Height = 48
Img_Channels = 1

In [28]:
model = build_UNet((Img_Width, Img_Height, Img_Channels),(Img_Width, Img_Height, Img_Channels))
model2 = tf.keras.models.clone_model(model)
# model.load_weights('UNet_JND_EOphtha.h5')

In [30]:
model.predict([tf.zeros([1,48,48,1]), tf.zeros([1,48,48,1])])

1/1 [==============================] - 0s 470ms/step


[array([[[[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]],
 
         [[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]],
 
         [[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]],
 
         ...,
 
         [[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]],
 
         [[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]],
 
         [[0.5],
          [0.5],
          [0.5],
          ...,
          [0.5],
          [0.5],
          [0.5]]]], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
# Reading the Training & Testing Images from Google Drive ..

datapath = "EOphtha_2L_Dataset\Images"
Training_imgs = sorted(glob(os.path.join(datapath, "Training_Set", "*jpg")))

datapath = "EOphtha_2L_Dataset\Ground_Truths"
Training_maps = sorted(glob(os.path.join(datapath, "Training_Set", "*png")))

In [ ]:
# Random Shuffling of Images & Maps Accordingly
c = list(zip(Training_imgs, Training_maps))
random.shuffle(c)
Training_imgs, Training_maps = zip(*c)

In [ ]:
# Just for Checking ..
print(len(Training_imgs))
print(len(Training_maps))

num = random.randint(0, len(Training_imgs))
print(Training_imgs[num])
print(Training_maps[num])

In [ ]:
# Making Training Images ready ..

length = len(Training_imgs)
train_X = np.zeros((length, Img_Width, Img_Height, Img_Channels), dtype=np.uint8)

for n in range(length):
    image = cv2.imread(Training_imgs[n], 0)

    image = np.expand_dims(image, axis=-1)
    train_X[n] = image

In [ ]:
# Making Segmentation Map (Ground Truth) for the Training Images ready ..

length = len(Training_maps)
train_Y = np.zeros((length, Img_Width, Img_Height, Img_Channels), dtype=float)

for n in range(length):
    seg_map = cv2.imread(Training_maps[n], 0)

    if np.max(seg_map)==0:
        seg_map = seg_map
    else:
        seg_map = seg_map / 255.0

    seg_map = seg_map > 0.5
    seg_map = np.expand_dims(seg_map, axis=-1)
    train_Y[n] = seg_map

In [ ]:
# Seperating Training and Validation Datasets
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.20, random_state=42)

In [ ]:
# Printing the Info ..
print('Training Set Details')
print(train_X.shape)
print(train_Y.shape)
print('Validation Set Details')
print(val_X.shape)
print(val_Y.shape)

In [ ]:
print("Unique Values of Training Masks are ", np.unique(train_Y))
print("Unique Values of Validation Masks are ", np.unique(val_Y))

In [ ]:
# Counting MA and nonMA images in training dataset
var0 = 0
var1 = 0
for i in range(len(train_Y)):
    img = train_Y[i]
    img = np.squeeze(img)
    if(np.max(img) != 0):
        var1 = var1 + 1
    else:
        var0 = var0 + 1
print("Train MA Images - {}".format(var1))
print("Train nonMA Images - {}".format(var0))

In [ ]:
# Counting MA and nonMA images in valiation dataset
var0 = 0
var1 = 0
for i in range(len(val_Y)):
    img = val_Y[i]
    img = np.squeeze(img)
    if(np.max(img) != 0):
        var1 = var1 + 1
    else:
        var0 = var0 + 1
print("Val MA Images - {}".format(var1))
print("Val nonMA Images - {}".format(var0))

In [ ]:
# (For Training DataSet) --> Displaying Fundus Image and its Ground Truth (Segmentation Map)

num = random.randint(0, len(train_X))   # Random Number
# Printing the size of Image
print(train_X[num].shape)
fundus_img = train_X[num]

# Printing the size of Mask
print(train_Y[num].shape)
img_mask = train_Y[num]

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(np.squeeze(fundus_img), cmap="gray")
ax1.grid(False)
ax2.imshow(np.squeeze(img_mask), cmap="gray")
ax2.grid(False)
plt.show()

In [ ]:
# (For Validation DataSet) --> Displaying Fundus Image and its Ground Truth (Segmentation Map)

num = random.randint(0, len(val_X))   # Random Number
# Printing the size of Image
print(val_X[num].shape)
fundus_img = val_X[num]

# Printing the size of Mask
print(val_Y[num].shape)
img_mask = val_Y[num]

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(np.squeeze(fundus_img), cmap="gray")
ax1.grid(False)
ax2.imshow(np.squeeze(img_mask), cmap="gray")
ax2.grid(False)
plt.show()

In [ ]:
# some parameters initializing ..
initial_LR = 1e-3
num_epochs = 150
wdecay = (initial_LR / num_epochs)
bs = 64

In [ ]:
# Function for Sensitivity & Specificity

def ce_loss(y_true, y_pred, maxis):
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    term_1 = y_true * K.log(y_pred + K.epsilon())

    ## Calculating ATT loss .. due to MA Part
    cnt = K.sum(K.cast(K.equal(maxis, 0), float))
    att_loss1 = K.sum(1 - maxis) - cnt

    bce_loss = -K.mean(term_0 + term_1)                               # This is Binary Cross Entropy Loss
    att_loss = (att_loss1 / bs)  # This is Attention Deviation Loss ('This loss concentrates on NonMA Regions')
    out = ((att_loss*att_loss) + (bce_loss*bce_loss)) / (att_loss + bce_loss)  # Total Loss is Contra Harmonic Mean of BCE_Loss & Attention Loss
    return out, bce_loss, att_loss

def acc_met(y_true,y_pred):
    out = K.mean(K.equal(y_true, y_pred))
    return out

def sensitivity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_pred = 1 - y_pred_f
    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f * neg_y_pred)
    return tp / (tp+fn+K.epsilon())

def specificity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_true = 1 - y_true_f
    neg_y_pred = 1 - y_pred_f
    fp = K.sum(neg_y_true * y_pred_f)
    tn = K.sum(neg_y_true * neg_y_pred)
    return tn / (tn + fp + K.epsilon())

In [ ]:
# Compiling the Model ..

opt = Adam(learning_rate = initial_LR, decay=wdecay)
model = build_UNet((Img_Width, Img_Height, Img_Channels),(Img_Width, Img_Height, Img_Channels))
metrics_evaluated = [acc_met,
                     sensitivity,
                     specificity]
model.compile(optimizer=opt, loss=ce_loss, metrics=metrics_evaluated)
model.summary()

In [ ]:
####################################### CUSTOM TRAINING #########################################
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size=bs, shuffle=True):
        super().__init__()
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.key_array = np.arange(self.images.shape[0], dtype=np.uint32)
        self.on_epoch_end()

    def __len__(self):
        return len(self.key_array)//self.batch_size

    def __getitem__(self, index):
        keys = self.key_array[index*self.batch_size:(index+1)*self.batch_size]
        x = np.asarray(self.images[keys], dtype=np.float32)
        y = np.asarray(self.labels[keys], dtype=np.float32)
        return x, y

    def on_epoch_end(self):
        if self.shuffle:
            self.key_array = np.random.permutation(self.key_array)

In [ ]:
generator = DataGenerator(images=train_X, labels=train_Y, batch_size=bs, shuffle=True)
n_batches = len(generator)

In [ ]:
# Initializing the variables

bceloss_train = np.zeros(shape=(num_epochs,), dtype=float)
attloss_train = np.zeros(shape=(num_epochs,), dtype=float)
totalloss_train = np.zeros(shape=(num_epochs,), dtype=float)
acc_train  = np.zeros(shape=(num_epochs,), dtype=float)
sens_train = np.zeros(shape=(num_epochs,), dtype=float)
spec_train = np.zeros(shape=(num_epochs,), dtype=float)

loss_val = np.zeros(shape=(num_epochs,), dtype=float)
acc_val  = np.zeros(shape=(num_epochs,), dtype=float)
sens_val = np.zeros(shape=(num_epochs,), dtype=float)
spec_val = np.zeros(shape=(num_epochs,), dtype=float)

threshold = 0.50 # Defining the Threshold

In [ ]:
for epoch in range(num_epochs):

    epoch_bceloss_avg = tf.keras.metrics.Mean() # Keeping track of the training loss
    epoch_attloss_avg = tf.keras.metrics.Mean() # Keeping track of the training loss
    epoch_totalloss_avg = tf.keras.metrics.Mean() # Keeping track of the training loss
    epoch_acc_avg  = tf.keras.metrics.Mean() # Keeping track of the training accuracy
    epoch_sens_avg = tf.keras.metrics.Mean() # Keeping track of the training sensitivity
    epoch_spec_avg = tf.keras.metrics.Mean() # Keeping track of the training specificity

    print('==== Epoch {}/{} ===='.format(epoch+1, num_epochs))

    for batch in tqdm(range(n_batches)):

        x, y = generator[batch]

        with tf.GradientTape() as tape: # Forward pass
            y_p, maxi = model([x, y], training=True)
            loss, bce_loss, att_loss = ce_loss(y_true=y, y_pred=y_p, maxis=maxi)

        grad = tape.gradient(loss, model.trainable_variables) # Backpropagation
        opt.apply_gradients(zip(grad, model.trainable_variables)) # Update network weights

        epoch_bceloss_avg(bce_loss)
        epoch_attloss_avg(att_loss)
        epoch_totalloss_avg(loss)
        y_pt = K.cast((y_p>threshold),float)
        epoch_acc_avg(acc_met(y_true=y, y_pred=y_pt))
        epoch_sens_avg(sensitivity(y_true=y, y_pred=y_pt))
        epoch_spec_avg(specificity(y_true=y, y_pred=y_pt))

    generator.on_epoch_end()

    # Training Predictions
    bceloss_train[epoch] = epoch_bceloss_avg.result()
    attloss_train[epoch] = epoch_attloss_avg.result()
    totalloss_train[epoch] = epoch_totalloss_avg.result()
    acc_train[epoch] = epoch_acc_avg.result()
    sens_train[epoch] = epoch_sens_avg.result()
    spec_train[epoch] = epoch_spec_avg.result()

    # Validation predictions
    y_p, maxi  = model.predict([val_X, val_Y], verbose=False)

    loss_val[epoch],_,_ = ce_loss(y_true=val_Y, y_pred=y_p, maxis=maxi)
    y_pt = K.cast((y_p>threshold),"double")
    acc_val[epoch] = acc_met(y_true=val_Y, y_pred=y_pt)
    sens_val[epoch] = sensitivity(y_true=val_Y, y_pred=y_pt)
    spec_val[epoch] = specificity(y_true=val_Y, y_pred=y_pt)

    print("bce_loss: {:.4f} - att_loss: {:.4f} - total_loss: {:.4f} - accuracy: {:.4f} - sensitivity: {:.4f} - specificity: {:.4f} - val_loss: {:.4f} - val_accuracy: {:.4f} - val_sensitivity: {:.4f} - val_specificity: {:.4f}".format(bceloss_train[epoch], attloss_train[epoch], totalloss_train[epoch], acc_train[epoch], sens_train[epoch], spec_train[epoch], loss_val[epoch], acc_val[epoch], sens_val[epoch], spec_val[epoch]))

In [ ]:
# plotting Training, Validation - Accuracy, Loss, Sensitivity, Specificity, AUC
N = num_epochs
plt.style.use("ggplot")

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(18, 5))

ax1.plot(np.arange(0,N), acc_val, label = "val_acc")
ax1.plot(np.arange(0,N), acc_train, label = "train_acc")
ax1.set_title("Train & Val Accuracy")
ax1.set(xlabel = "Epoch Number", ylabel = "Accuracy")
ax1.legend(loc = "lower right")

ax2.plot(np.arange(0,N), loss_val, label = "val_loss")
ax2.plot(np.arange(0,N), totalloss_train, label = "train_loss")
ax2.set_title("Train & Val Loss")
ax2.set(xlabel = "Epoch Number", ylabel = "Loss")
ax2.legend(loc = "upper right")

ax3.plot(np.arange(0,N), sens_val, label = "val_sensitivity")
ax3.plot(np.arange(0,N), sens_train, label = "train_sensitivity")
ax3.set_title("Train & Val Sensitivity")
ax3.set(xlabel = "Epoch Number", ylabel = "Sensitivity")
ax3.legend(loc = "lower right")

ax4.plot(np.arange(0,N), spec_val, label = "val_specificity")
ax4.plot(np.arange(0,N), spec_train, label = "train_specificity")
ax4.set_title("Train & Val Specificity")
ax4.set(xlabel = "Epoch Number", ylabel = "Specificity")
ax4.legend(loc = "lower right")

fig.tight_layout()
plt.savefig("Metrics_Plot.png")
plt.show()

In [ ]:
# Saving the Model
model.save("UNet_EOphtha_simAM_Attloss_T1.h5")

In [ ]:
########################################################### END OF THE CODE ###########################################################